In [10]:
import requests
import os
from dotenv import load_dotenv
from fuzzywuzzy import process
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from datetime import datetime
import pytz
import pandas as pd

In [11]:
load_dotenv()
odoo_url = "https://dev-unitiva.odoo.com/"
username = os.getenv("user")
pw = os.getenv('pw')
db = os.getenv('db')
api_key= os.getenv('openai_key')

llm = ChatOpenAI(
    api_key=api_key,
    model='gpt-4o',
    temperature=0
)

# Modelli che richiedono employee_id
modelli_con_employee_id = [
    "hr.leave",          # Richieste di ferie
    "hr.attendance",     # Presenze giornaliere
    "hr.contract",       # Contratti dipendenti
    "hr.expense",        # Note spese
    "account.analytic.line",  # Fogli ore legati ai dipendenti
    "hr.payslip",        # Buste paga
    "hr.skill",          # Competenze del dipendente
    "hr.employee.public", # Record dipendenti pubblici (ridotto)
]

# Modelli che richiedono uid
modelli_con_uid = [
    "res.users",         # Dati relativi all'utente
    "calendar.event",    # Eventi in calendario assegnati all'utente
    "mail.activity",     # Attività pianificate dall'utente
    "res.partner",       # Partner e contatti collegati all'utente
    "crm.lead",          # Opportunità di vendita (CRM)
    "helpdesk.ticket",  # Ticket di supporto assegnati
    'project.project',
    'project.task'
]

In [12]:
#METODO PER EFFETTURA AUTENTICAZIONE SU ODOO (OK)
def autenticazione():
    url_auth = f"{odoo_url}/web/session/authenticate"
    payload_auth = {
        "jsonrpc": "2.0",
        "params": {
            "db":db,
            "login": username,
            "password": pw
        }
    }
    #effettua richiesta
    session = requests.Session()
    response_auth = session.post(url_auth, json=payload_auth)
    response_auth.raise_for_status()
    result = response_auth.json()
    try:
        uid = result['result']['uid']
    except Exception as e:
        return "Autenticazione fallita"
    
    return uid,session

In [13]:
modello = 'project.project'

In [14]:
uid, session = autenticazione()
    
    # Ricerca degli ID dei record
url_search = f"{odoo_url}/web/dataset/call_kw/{modello}/search_read"

# Aggiungi user_id ai filtri, se necessario
#filtri.append(["user_id", "=", uid])
filtri = ['|', 
    ["user_id", "=", uid],
    ["member_ids", "in", [uid]]
]
payload_search = {
    "jsonrpc": "2.0",
    "params": {
        "model": modello,
        "method": "search_read",
        "args": [filtri],
        "kwargs": {
            "fields": ["id","name"],
            "limit": 0  # Restituisce almeno un record
        }
    }
}


In [15]:
print(payload_search)

{'jsonrpc': '2.0', 'params': {'model': 'project.project', 'method': 'search_read', 'args': [['|', ['user_id', '=', 66], ['member_ids', 'in', [66]]]], 'kwargs': {'fields': ['id', 'name'], 'limit': 0}}}


In [16]:
try:
    response_search = session.post(url_search, json=payload_search)
    response_search.raise_for_status()
    results = response_search.json().get('result', [])
    
    # Verifica se ci sono risultati
    if results:
        record_ids = [record['id'] for record in results]
        print(record_ids)
    else:
        print("Nessun record trovato con i filtri forniti",payload_search)

except Exception as e:
    print(f"Errore durante la ricerca degli ID: {e}")

Nessun record trovato con i filtri forniti {'jsonrpc': '2.0', 'params': {'model': 'project.project', 'method': 'search_read', 'args': [['|', ['user_id', '=', 66], ['member_ids', 'in', [66]]]], 'kwargs': {'fields': ['id', 'name'], 'limit': 0}}}


In [22]:
modello = 'project.project'
url_search = f"{odoo_url}/web/dataset/call_kw/{modello}/search_read"

# Prima proviamo a cercare anche i progetti inattivi
payload_search = {
    "jsonrpc": "2.0",
    "params": {
        "model": modello,
        "method": "search_read",
        "args": [["|", ["active", "=", True], ["active", "=", False]]],  # Cerca sia attivi che inattivi
        "kwargs": {
            "fields": ["id", "name", "user_id", "active", "partner_id"],
            "limit": 0
        }
    }
}

try:
    print("\nRicerca di tutti i progetti (attivi e inattivi):")
    response_search = session.post(url_search, json=payload_search)
    response_search.raise_for_status()
    results = response_search.json().get('result', [])
    
    if results:
        print(f"\nTrovati {len(results)} progetti:")
        for project in results:
            print(project)
    else:
        print("\nNessun progetto trovato")
        
    # Proviamo anche a vedere le regole di accesso per il modello
    modello_access = 'ir.model.access'
    payload_access = {
        "jsonrpc": "2.0",
        "params": {
            "model": modello_access,
            "method": "search_read",
            "args": [[["model_id.model", "=", "project.project"]]],
            "kwargs": {
                "fields": ["name", "perm_read", "perm_write", "perm_create", "perm_unlink"],
                "limit": 0
            }
        }
    }
    
    print("\nVerifica permessi sul modello project.project:")
    response_access = session.post(url_search, json=payload_access)
    print(response_access.json().get('result', []))

except Exception as e:
    print(f"Errore durante la ricerca: {e}")


Ricerca di tutti i progetti (attivi e inattivi):

Trovati 64 progetti:
{'id': 15, 'name': 'AMMINISTRAZIONE', 'user_id': [29, 'Maura Cimmino'], 'active': True, 'partner_id': False}
{'id': 14, 'name': 'COMUNICAZIONE', 'user_id': [33, 'Stefania Scolamiero'], 'active': True, 'partner_id': False}
{'id': 55, 'name': 'COT - Rif. Alessio Boerio - S00075', 'user_id': False, 'active': True, 'partner_id': [716, 'FIVEN S.p.A.']}
{'id': 7, 'name': 'CROCE ROSSA', 'user_id': [24, 'Livia Buonvino'], 'active': True, 'partner_id': [716, 'FIVEN S.p.A.']}
{'id': 11, 'name': 'Consulenza Progetto Odoo - S00011 - [Progetto a corpo] Sviluppo progetto a corpo (Proj)', 'user_id': [31, 'Raffaele Amalfitano'], 'active': True, 'partner_id': [690, 'Allcore Spa']}
{'id': 80, 'name': 'DG Biotech', 'user_id': [33, 'Stefania Scolamiero'], 'active': True, 'partner_id': False}
{'id': 20, 'name': 'DIMORADUOMO', 'user_id': [24, 'Livia Buonvino'], 'active': True, 'partner_id': False}
{'id': 18, 'name': 'FEDRIGONI PRICING T